## Модель распознавания эмоций из речи
 
 Модель распознавания эмоций из речи с использованием библиотек librosa и sklearn и набора данных RAVDESS.

В этом проекте Python мы будем использовать библиотеки **librosa**, **soundfile** и **sklearn** (среди прочих) для построения модели с использованием MLPClassifier. Это позволит распознавать эмоции из звуковых файлов. 
 

**Последовательность выполнения задачи**

* загрузим данные
* извлекем из них объекты
* разделим набор данных на обучающие и тестовые наборы
* инициализируем MLP-классификатор и обучим модель 
* рассчитаем точность нашей модели.

In [16]:
!pip install librosa soundfile

In [17]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [18]:
#Connect your Drive with Colab
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [29]:
#Unzip the file contents
!unzip '/content/drive/MyDrive/speech-emotion-recognition-ravdess-data.zip'

Archive:  /content/drive/MyDrive/speech-emotion-recognition-ravdess-data.zip
replace Actor_01/03-01-01-01-01-01-01.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Определение функции extract_feature для извлечения функций mfcc, chroma и mel из звукового файла. Эта функция принимает 4 параметра - имя файла и три логических параметра для трех функций:

* mfcc:  представляет собой кратковременный спектр мощности звука (переходы в звуковых тонах)
* chroma: Относится к 12 различным классам высоты тона
* mel: Частота Спектрограммы Mel

Открываем звуковой файл с помощью soundfile.SoundFile используя with-as, далее файл автоматически закрывается, как только мы закончим. прочтем его и назовем X. Кроме того, получим частоту дискретизации. Если chroma истинна, получим преобразование Фурье X.

Пусть результатом будет пустой массив numpy. Теперь для каждого параметра функции, сделаем вызов соответствующей функции из librosa.feature (например - librosa.feature.mfcc для mfcc) и получим среднее значение. Вызовите функцию hstack() из numpy значение функции и сохраним в result. hstack() соединят массивы по горизонтали. Затем вернем результ.

In [31]:
# определение функции extract_feature для извлечения функций mfcc, chroma и mel из звукового файла
def extract_feature(file_name, mfcc, chroma, mel):
  with soundfile.SoundFile(file_name) as sound_file:
    X = sound_file.read(dtype="float32")
    sample_rate=sound_file.samplerate
    if chroma:
      stft=np.abs(librosa.stft(X))
    result=np.array([])
    if mfcc:
      mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
      result=np.hstack((result, mfccs))
    if chroma:
      chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
      result=np.hstack((result, chroma))
    if mel:
      mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
      result=np.hstack((result, mel))
  return result

In [32]:
# определим словарь эмоций 
emotions = {
    '01':'neutral',
    '02':'calm',
    '03':'happy',
    '04':'sad',
    '05':'angry',
    '06':'fearful',
    '07':'disgust',
    '08':'surprised'
}

# рассматриваемые эмоции
observed_emotions = ['calm', 'happy', 'fearful', 'disgust']

Теперь загрузим данные с помощью функции load_data() – она принимает в качестве параметра относительный размер тестового набора. x и y-пустые списки;
Будем использовать функцию glob() из модуля glob, чтобы получить все пути к звуковым файлам в нашем наборе данных.
Получите базовое имя файла, эмоцию, разделив имя ‘-’ и извлекая третье значение

Используя наш словарь эмоций, число соответствует эмоции, и наша функция проверяет, есть ли эта эмоция в нашем списке observed_emotions; если нет, она переходит к следующему файлу. Он вызывает extract_feature и сохраняет то, что возвращается в "feature". Затем он добавляет функцию к x, а эмоцию-к y. Итак, список x содержит черты, а y-эмоции. Мы вызываем функцию train_test_split с этими данными, размером теста и случайным значением состояния и возвращаем его.

In [33]:
# загрузим данные и извлеките функции для каждого звукового файла
def load_data(test_size = 0.2):
  x, y = [], []
  for folder in glob.glob('/content/Actor_*'):
    print(folder)
    for file in glob.glob(folder + '/*.wav'):
      file_name = os.path.basename(file)
      emotion = emotions[file_name.split('-')[2]]
      if emotion not in observed_emotions:
        continue
      feature = extract_feature(file, mfcc = True, chroma = True, mel = True)
      x.append(feature)
      y.append(emotion)
  return train_test_split(np.array(x), y, test_size = test_size, random_state = 9)

In [23]:
x_train,x_test,y_train,y_test=load_data(test_size=0.2)

/content/Actor_17
/content/Actor_03
/content/Actor_08
/content/Actor_23
/content/Actor_19
/content/Actor_09
/content/Actor_10
/content/Actor_14
/content/Actor_05
/content/Actor_21
/content/Actor_01
/content/Actor_15
/content/Actor_11
/content/Actor_18
/content/Actor_13
/content/Actor_02
/content/Actor_22
/content/Actor_04
/content/Actor_07
/content/Actor_24
/content/Actor_20
/content/Actor_16
/content/Actor_06
/content/Actor_12


In [34]:
print((x_train.shape[0], x_test.shape[0]))
print(f'Features extracted: {x_train.shape[1]}')

(614, 154)
Features extracted: 180


Теперь давайте инициализируем MLPClassifier. Это многослойный персептронный классификатор, он оптимизирует функцию логарифмических потерь с помощью LBFGS или стохастического градиентного спуска. В отличие от SVM или наивного Байеса, MLPClassifier имеет внутреннюю нейронную сеть для целей классификации. Это модель прямой ANN.

In [35]:
# инициализируем Multi Layer Perceptron Classifier
model = MLPClassifier(alpha = 0.01, batch_size = 256, epsilon = 1e-08, hidden_layer_sizes = (300,), learning_rate = 'adaptive', max_iter = 500)

In [36]:
model.fit(x_train, y_train)

MLPClassifier(activation='relu', alpha=0.01, batch_size=256, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [37]:
y_pred = model.predict(x_test)

In [38]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 76.62%


## Вывод
  В этом мини-проекте с помощью MLPClassifier и librosa в представлении пространственных объектов, трансформировании и кодировании последовательностей мне удается достичь точности >75% на тестовом наборе данных RAVDESS.